In [60]:
import os 
import requests 
from urllib.request import urlopen
from openai import OpenAI
from bs4 import BeautifulSoup
from IPython.display import Markdown, display
from dotenv import load_dotenv

In [61]:
load_dotenv()
api_key=os.getenv("OPENAI_API_KEY")

if not api_key:
    print("No API key was found - Please try to identify & fix!")
elif not api_key.startswith !="sk-proj-":
    print("An API key was found, but it doesn't start sk-proj-; please check you're using the right key.")
elif not api_key.split()!=api_key:
    print("An API key was found, but it looks like it might have space or tab characters at the start or end - please remove them.")
else:
    print("API key found and looks good so far!")

API key found and looks good so far!


In [62]:
openai = OpenAI()

In [63]:
headers={
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

In [64]:
re_sess=requests.Session()

In [66]:
myurl = input("Enter a website URL: ")

try:
    response = requests.get(myurl,headers=headers,timeout=10)
    print(response.status_code)
except requests.exceptions.RequestException as e:
    print(f"❌ Failed to fetch website: {e}")
    myurl=None

Enter a website URL:  https://cnn.com


200


In [67]:
soup=BeautifulSoup(response.content,'html.parser')

In [68]:
title = soup.title.string if soup.title else "No title found."

In [69]:
for irrelevant in soup.body(["script", "style", "img", "input"]):
    irrelevant.decompose()
text = soup.body.get_text(separator="\n", strip=True)

In [70]:
system_prompt = """You are an assistant that summarizes website content.
- Provide a short, clear summary (3–5 sentences max).
- Focus only on the main ideas, purpose, or offerings of the website.
- Ignore navigation links, ads, boilerplate text, and repetitive content.
- If the text looks like a blog/article, summarize it as an article.
- If the text is company/product info, summarize it as a business overview.
- Respond in clean markdown format with no extra commentary."""

In [71]:
def user_prompt_for(myurl):
    user_prompt = "" 
    user_prompt += f"You are currently on the website titled {title}\n"
    user_prompt += "\nThe contents of this website is as follows; \
please provide a short summary of this website in markdown. \
If it includes news or announcements or some keynotes, then summarize these too.\n\n"
    user_prompt += text
    return user_prompt

In [72]:
def messages_for(myurl):
    return [
        {'role':'system','content':system_prompt},
        {'role':'user','content':user_prompt_for(myurl)}
    ]

In [79]:
def summarize(myurl):
    response = openai.chat.completions.create(
        model = "gpt-5-nano",
        messages = messages_for(myurl)
    )
    return response.choices[0].message.content

In [80]:
summarize(myurl)

"CNN's Breaking News, Latest News and Videos site provides global and US news coverage across categories such as World, Politics, Business, Health, Entertainment, Style, Travel, Sports, Science, and Climate. The site features articles, in-depth analyses, investigations, and multimedia content—including video and live TV—covering major events like conflicts and elections. Readers can watch CNN videos, listen to podcasts, and sign in to personalize feeds and newsletters. The homepage also spotlights special reports and programs such as CNN Heroes and Call to Earth, highlighting a mix of hard news and long-form storytelling."

In [77]:
def display_better(myurl):
    summary = summarize(myurl)
    display(Markdown(summary))

In [78]:
display_better(myurl)

CNN's Breaking News, Latest News and Videos page is a comprehensive news portal offering around-the-clock global coverage across US and world politics, business, health, entertainment, science, climate, and more. It provides live TV and a wide range of video and audio content, including CNN10, CNN Fast, podcasts, and video reports. The site highlights major stories and in-depth analysis on conflicts (Gaza, Ukraine-Russia, Israel-Hamas), elections, technology and other emerging topics, with continually updated headlines and features like “Best of CNN” and “Call to Earth.” It also promotes personalized content through user accounts, newsletters, topics you follow, and ML-powered content recommendations to surface relevant articles and videos.